In [ ]:
# @title Cell 1: Setup & Imports
import subprocess
import os
import json
import glob
from google.colab import files
import shutil

# Ensure ffmpeg is installed
!apt-get install -y ffmpeg

print("✅ Setup Complete.")

# Black & White

In [ ]:
# @title Cell 2: Black & White - Upload Video
print("📂 Please upload the video for the 'Black & White' task.")
uploaded = files.upload()

if len(uploaded) == 0:
    print("❌ No file uploaded.")
    bw_input_filename = None
else:
    bw_input_filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {bw_input_filename}")

In [ ]:
# @title Cell 3: Black & White - Process Video
# Logic adapted from replace_blackandwhite.py

def get_video_info(video_path):
    cmd = [
        'ffprobe', '-v', 'quiet', '-print_format', 'json',
        '-select_streams', 'v:0',
        '-show_entries', 'stream=width,height,avg_frame_rate,codec_name',
        video_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    info = json.loads(result.stdout)
    stream = info['streams'][0]
    width = stream['width']
    height = stream['height']

    fps_str = stream.get('avg_frame_rate', '30/1')
    if '/' in fps_str and fps_str.split('/')[1] != '0':
        num, den = map(int, fps_str.split('/'))
        fps = num / den if den != 0 else 30.0
    else:
        fps = 30.0
    return width, height, fps

def process_black_white(input_file):
    if not input_file or not os.path.exists(input_file):
        print("❌ Input file not found.")
        return

    print(f"⚙️ Processing {input_file}...")
    try:
        width, height, fps = get_video_info(input_file)

        # Generate Output Filename: [original]_replaced.mp4
        base_name = os.path.splitext(os.path.basename(input_file))[0]
        output_path = f"{base_name}_replaced.mp4"

        print(f"📊 Detected: {width}x{height}, {fps:.2f} FPS")
        print(f"💾 Output will be: {output_path}")

        # Step 1: Create a single black frame
        black_cmd = [
            'ffmpeg', '-f', 'lavfi',
            '-i', f'color=black:size={width}x{height}:rate={fps}',
            '-frames:v', '1', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-y',
            'temp_black.mp4'
        ]
        subprocess.run(black_cmd, check=True, capture_output=True)

        # Step 2: Extract all frames starting from frame 1 (skip frame 0)
        rest_cmd = [
            'ffmpeg', '-i', input_file,
            '-vf', 'select=gte(n\\,1),setpts=N/FRAME_RATE/TB',
            '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-r', str(fps), '-y',
            'temp_rest.mp4'
        ]
        subprocess.run(rest_cmd, check=True, capture_output=True)

        # Step 3: Concatenate black frame + rest of video
        concat_cmd = [
            'ffmpeg', '-i', 'temp_black.mp4', '-i', 'temp_rest.mp4',
            '-filter_complex', '[0:v][1:v]concat=n=2:v=1:a=0[v]',
            '-map', '[v]', '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
            '-r', str(fps), '-y', output_path
        ]
        subprocess.run(concat_cmd, check=True, capture_output=True)

        # Step 4: Copy audio from original (if exists)
        probe_cmd = [
            'ffprobe', '-v', 'quiet', '-select_streams', 'a:0',
            '-show_entries', 'stream=codec_type', '-of', 'csv=p=0', input_file
        ]
        audio_check = subprocess.run(probe_cmd, capture_output=True, text=True)

        if 'audio' in audio_check.stdout.lower():
            print("🔊 Audio detected, merging...")
            final_with_audio = f"{base_name}_temp_audio.mp4"
            add_audio_cmd = [
                'ffmpeg', '-i', output_path, '-i', input_file,
                '-c:v', 'copy', '-c:a', 'aac',
                '-map', '0:v:0', '-map', '1:a:0', '-y',
                final_with_audio
            ]
            subprocess.run(add_audio_cmd, check=True, capture_output=True)
            os.replace(final_with_audio, output_path)
        else:
            print("🔇 No audio detected.")

        # Cleanup
        for temp in ['temp_black.mp4', 'temp_rest.mp4']:
            if os.path.exists(temp): os.remove(temp)

        print(f"✅ Saved as: {output_path}")

    except Exception as e:
        print(f"❌ Error: {e}")

# Run processing
if 'bw_input_filename' in globals() and bw_input_filename:
    process_black_white(bw_input_filename)
else:
    print("❌ Please run Cell 2 first.")

# Grey

In [ ]:
# @title Cell 4: Grey - Upload Source Video (Step 1)
print("📂 Step 1: Please upload 'video_source' (Frame Donor).")
uploaded = files.upload()

if len(uploaded) == 0:
    print("❌ No file uploaded.")
    grey_source_filename = None
else:
    grey_source_filename = list(uploaded.keys())[0]
    print(f"✅ Source Uploaded: {grey_source_filename}")

In [ ]:
# @title Cell 5: Grey - Upload Target Video (Step 2)
print("📂 Step 2: Please upload 'video_target' (Video to Edit).")
uploaded = files.upload()

if len(uploaded) == 0:
    print("❌ No file uploaded.")
    grey_target_filename = None
else:
    grey_target_filename = list(uploaded.keys())[0]
    print(f"✅ Target Uploaded: {grey_target_filename}")

In [ ]:
# @title Cell 6: Grey - Process Frame Swap
# Logic adapted from replace_grey.py

def get_video_info_grey(video_path):
    cmd = [
        'ffprobe', '-v', 'quiet', '-print_format', 'json',
        '-show_streams', '-select_streams', 'v:0', video_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    info = json.loads(result.stdout)
    stream = info['streams'][0]

    if 'nb_frames' in stream:
        total_frames = int(stream['nb_frames'])
    else:
        duration = float(stream.get('duration', 0))
        fps_str = stream.get('avg_frame_rate', '30/1')
        if '/' in fps_str:
            num, den = map(int, fps_str.split('/'))
            fps_val = num / den if den != 0 else 30.0
        else:
            fps_val = 30.0
        total_frames = int(duration * fps_val)

    fps = stream['avg_frame_rate']
    width = stream['width']
    height = stream['height']
    return total_frames, fps, width, height

def process_grey(source_file, target_file):
    if not source_file or not target_file:
        print("❌ Input files not found.")
        return
    if not os.path.exists(source_file) or not os.path.exists(target_file):
        print("❌ One or both files do not exist.")
        return

    print("⚙️ Processing Grey Task...")
    try:
        # Get Source Info
        total_frames, _, _, _ = get_video_info_grey(source_file)
        target_frame_index = total_frames - 24

        if target_frame_index < 0:
            print(f"❌ Error: Source has only {total_frames} frames. Need at least 24.")
            return

        print(f"📊 Source Frames: {total_frames} | Extracting Index: {target_frame_index}")

        # Extract Target Frame from Source
        temp_frame = "temp_replacement_frame.png"
        cmd_extract = [
            'ffmpeg', '-y', '-i', source_file,
            '-vf', f'select=eq(n\\,{target_frame_index})',
            '-vframes', '1', temp_frame
        ]
        subprocess.run(cmd_extract, check=True, capture_output=True)

        # Get Target Info
        _, fps, width, height = get_video_info_grey(target_file)

        # Generate Output Filename: [target_original]_replaced.mp4
        base_name = os.path.splitext(os.path.basename(target_file))[0]
        output_path = f"{base_name}_replaced.mp4"

        print(f"📊 Target Res: {width}x{height} | FPS: {fps}")
        print(f"💾 Output will be: {output_path}")

        # Replace First Frame using Overlay
        filter_complex = (
            f"[1:v]scale={width}:{height}:force_original_aspect_ratio=decrease,"
            f"pad={width}:{height}:(ow-iw)/2:(oh-ih)/2[fg];"
            f"[0:v][fg]overlay=enable='eq(n,0)'[out]"
        )

        cmd_replace = [
            'ffmpeg', '-y',
            '-i', target_file,
            '-i', temp_frame,
            '-filter_complex', filter_complex,
            '-map', '[out]',
            '-map', '0:a?',
            '-c:v', 'libx264',
            '-preset', 'fast',
            '-crf', '23',
            '-r', fps,
            output_path
        ]
        subprocess.run(cmd_replace, check=True, capture_output=True)

        # Cleanup
        if os.path.exists(temp_frame): os.remove(temp_frame)

        print(f"✅ Saved as: {output_path}")

    except Exception as e:
        print(f"❌ Error: {e}")

# Run processing
if 'grey_source_filename' in globals() and 'grey_target_filename' in globals():
    process_grey(grey_source_filename, grey_target_filename)
else:
    print("❌ Please run Cell 4 and Cell 5 first.")

# Download

In [ ]:
# @title Cell 7: Download All Processed Videos
import zipfile

# Find all files ending with _replaced.mp4
processed_files = glob.glob("*_replaced.mp4")

if not processed_files:
    print("❌ No output files found to download.")
else:
    print(f"📦 Found {len(processed_files)} processed files:")
    for f in processed_files:
        print(f"  - {f}")

    # Create a zip archive with specific files
    zip_name = "processed_videos.zip"
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in processed_files:
            zipf.write(file)

    # Trigger Download
    files.download(zip_name)
    print("✅ Download started!")